In [1]:
import os
os.environ['AWS_PROFILE']     = 'osn-renci'
os.environ['AWS_S3_ENDPOINT'] = 'https://renc.osn.xsede.org'


In [2]:
import fsspec
import zarr
import rechunker
print("fsspec version: ", fsspec.__version__)
print("Zarr version: ", zarr.__version__)
print("ReChunker version: ", rechunker.__version__)

fsspec version:  2023.3.0+5.gbac7529
Zarr version:  2.14.0
ReChunker version:  0.5.1


In [3]:
import xarray as xr 
import intake
url = 'https://raw.githubusercontent.com/hytest-org/hytest/main/dataset_catalog/hytest_intake_catalog.yml'
cat = intake.open_catalog(url)
sampleData = cat['rechunking-tutorial-cloud'].to_dask()
sampleData

<xarray.Dataset>
Dimensions:     (feature_id: 400, time: 744)
Coordinates:
    elevation   (feature_id) float32 dask.array<chunksize=(400,), meta=np.ndarray>
  * feature_id  (feature_id) int32 3109 189899 239166 ... 947070134 1010003783
    gage_id     (feature_id) |S15 dask.array<chunksize=(400,), meta=np.ndarray>
    latitude    (feature_id) float32 dask.array<chunksize=(400,), meta=np.ndarray>
    longitude   (feature_id) float32 dask.array<chunksize=(400,), meta=np.ndarray>
    order       (feature_id) int32 dask.array<chunksize=(400,), meta=np.ndarray>
  * time        (time) datetime64[ns] 2000-07-01 ... 2000-07-31T23:00:00
Data variables:
    streamflow  (time, feature_id) float64 dask.array<chunksize=(256, 16), meta=np.ndarray>
    velocity    (time, feature_id) float64 dask.array<chunksize=(256, 16), meta=np.ndarray>
Attributes:
    TITLE:                OUTPUT FROM WRF-Hydro v5.2.0-beta2
    code_version:         v5.2.0-beta2
    featureType:          timeSeries
    model_configuration:  retrospective
    proj4:                +proj=lcc +units=m +a=6370000.0 +b=6370000.0 +lat_1...

In [4]:
fsw = fsspec.filesystem('file')
outfile = fsw.get_mapper(r"/tmp/output.zarr")
tmpfile = fsw.get_mapper(r"/tmp/tmpfile.zarr")

## Re-Chunking the Sample Data


In [5]:
# Numbers are *size* of the chunk. 
chunk_plan = {
    'streamflow': {'time': 744, 'feature_id': 1}, # all time records in one chunk for each feature_id
    'velocity': {'time': 744, 'feature_id': 1},
    'elevation': (400,),
    'gage_id': (400,),
    'latitude': (400,),
    'longitude': (400,),    
    'order': (400,),    
    'time': (744,),
    'feature_id': (400,)
}

With this plan, we can ask `rechunker` to re-write the data using the prescribed chunking pattern.


In [6]:
result = rechunker.rechunk(
    sampleData,
    chunk_plan,
    "2GB",
    outfile ,
    temp_store=tmpfile
)
_ = result.execute()

ReadOnlyError: object is read-only